In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/deep_learning_quantized')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import architectures
import network_training
import preprocessing

In [4]:
def read_data_from_csv(csv_file_name):
      data = pd.read_csv(csv_file_name)
      x_data = data.iloc[:, :-2].values
      y_data = data.iloc[:, -2].values
      subj = data.iloc[:, -1].values

      return x_data, y_data, subj

In [5]:
csv_file_name = '/content/drive/MyDrive/deep_learning_quantized/multimodal_data.csv'
labels = ('low', 'medium', 'high')
avg = True

features, targets, subj_data = read_data_from_csv(csv_file_name)
fs = 64

In [ ]:
# features = features[:, 0:4]
features = features[:, 0].reshape(-1, 1)
# features = features[:, 1:4]

# features = features[:, 5].reshape(-1, 1)
# features = features[:, 6:9]

In [7]:
params = {
    "seed" : 256,
    "learning_rate" : 0.002,
    "weight_decay" : 1e-6,
    "step_size" : 3,
    "gamma" : 0.85,
    "batch_size" : 128,
    "epochs" : 30,
    "num_resblocks" : 3,
    "resblock_id" : 0
}

params["window_size"] = fs * 8
params["overlap"] = params["window_size"] * 7//8

In [ ]:
scaled_data = preprocessing.scale_data(features, targets)

sliding_X_data, sliding_y_data = preprocessing.apply_sliding_window(scaled_data, targets, subj_data, params["window_size"], params["overlap"], avg)

X_data = sliding_X_data.astype(np.float32)
y_data = sliding_y_data.astype(np.uint8)

print(X_data.shape, y_data.shape)

In [9]:
params["num_channels"] = X_data.shape[2]
params["num_classes"] = len(labels)

In [19]:
from sklearn.model_selection import KFold
import keras.utils
import time

total_accuracy = []
total_f1_score = []
total_precision = []
total_recall = []
total_non_quantized_time = []
total_non_quantized_size = []

total_accuracy_quant = []
total_f1_score_quant = []
total_precision_quant = []
total_recall_quant = []
total_quantized_time = []
total_quantized_size = []

def training_loop(X_data, y_data, params, crossvalid = False):
  if crossvalid:
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state = params["seed"])
    data_splits = [(X_data[train_ind], X_data[val_ind], y_data[train_ind], y_data[val_ind]) for train_ind, val_ind in kf.split(X_data)]
  else:
    num_folds = 1
    X_train, X_val, y_train, y_val = network_training.split_data(X_data, y_data, train_size=0.8)
    data_splits = [(X_train, X_val, y_train, y_val)]

  for fold, (X_train_fold, X_val_fold, y_train_fold, y_val_fold) in enumerate(data_splits):
      print(f"Fold {fold+1}/{num_folds}")

      y_train_fold = keras.utils.to_categorical(y_train_fold, num_classes=3)
      y_val_fold = keras.utils.to_categorical(y_val_fold, num_classes=3)

      X_train_fold = np.expand_dims(X_train_fold, axis=-1)
      X_val_fold = np.expand_dims(X_val_fold, axis=-1)

      model, accuracy, f1_score, precision, recall, non_quantized_time = network_training.train_model(params, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
      if fold == 0: print(model.summary())

      tflite_quant_model = network_training.get_quantized_model(model)
      interpreter = network_training.get_tflite_interpreter(tflite_quant_model)
      accuracy_quant, f1_score_quant, precision_quant, recall_quant, quantized_time = network_training.evaluate_quantized_metrics(interpreter, X_val_fold, y_val_fold)

      non_quantized_model_size, quantized_model_size =  network_training.compare_model_sizes(tflite_quant_model, model)

      print(f"""Non-quantized: acc - {accuracy}, f1 - {f1_score}, prec - {precision}, rec - {recall}""")
      print(f"""Quantized: acc - {accuracy_quant}, f1 - {f1_score_quant}, prec - {precision_quant}, rec - {quantized_time}""")
      print(f"""Non-quantized time: {non_quantized_time}; Quantized time: {quantized_time} """)
      print(f"""Non-quantized size: {non_quantized_model_size}; Quantized size: {quantized_model_size} MB """)

      total_accuracy.append(accuracy)
      total_f1_score.append(f1_score)
      total_precision.append(precision)
      total_recall.append(recall)
      total_non_quantized_time.append(non_quantized_time)
      total_non_quantized_size.append(non_quantized_model_size)

      total_accuracy_quant.append(accuracy_quant)
      total_f1_score_quant.append(f1_score_quant)
      total_precision_quant.append(precision_quant)
      total_recall_quant.append(recall_quant)
      total_quantized_time.append(quantized_time)
      total_quantized_size.append(quantized_model_size)

  mean_accuracies = np.mean(total_accuracy)
  mean_f1_scores = np.mean(total_f1_score)
  mean_precisions = np.mean(total_precision)
  mean_recalls = np.mean(total_recall)
  mean_times = np.mean(total_non_quantized_time)
  mean_size = np.mean(total_non_quantized_size)

  mean_accuracies_quant = np.mean(total_accuracy_quant)
  mean_f1_scores_quant = np.mean(total_f1_score_quant)
  mean_precisions_quant = np.mean(total_precision_quant)
  mean_recalls_quant = np.mean(total_recall_quant)
  mean_times_quant = np.mean(total_quantized_time)
  mean_size_quant = np.mean(total_quantized_size)

  print(f"""Non-quantized:  acc - {mean_accuracies*100:.2f} F1 - {mean_f1_scores*100:.2f}
        prec - {mean_precisions*100:.2f} rec - {mean_recalls*100:.2f}
        time - {mean_times:.2f} size - {mean_size:.2f}""")

  print(f"""Quantized:  acc - {mean_accuracies_quant*100:.2f} F1 - {mean_f1_scores_quant*100:.2f}
        prec - {mean_precisions_quant*100:.2f} rec - {mean_recalls_quant*100:.2f}
        time - {mean_times_quant:.2f} size - {mean_size_quant}""")

In [ ]:
training_loop(X_data, y_data, params, True)